In [2]:
import json
import pandas as pd
import geopandas as gpd
from fuzzywuzzy import fuzz

with open('Data/unsupervised_places_interactions_ch3.json', 'r') as f:
    data = json.load(f)

json_df = pd.read_json('Data/unsupervised_places_interactions_ch3.json')

#display(data)
display(json_df)



,place,interaction,description
0,Jerusalem,visited,Visited and explored various localities in and...
1,Bethlehem,mentioned,Plans to visit Bethlehem and reflect on the bi...
2,Bethany,mentioned,Intends to visit Bethany and read about the re...
3,Gethsemane,mentioned,Looks forward to visiting Gethsemane to contem...
4,Calvary,mentioned,Aims to visit Calvary and reflect on the story...
5,Mount Olivet,mentioned,Anticipates visiting Mount Olivet to consider ...
6,Zion,observed,"Observed the hill of Zion, a significant locat..."
7,Akra,observed,Noted the hill of Akra while exploring Jerusalem.
8,Bezetha,observed,"Bezetha, one of the hills on which Jerusalem i..."
9,Moriah,observed,"Observed Moriah, another hill forming part of ..."


In [3]:
# Replace this with the path to your .shp file
shapefile_path = 'Data/landmarks_names.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Now you can work with the GeoDataFrame 'gdf' as you would with a Pandas DataFrame
gdf.head()  # This will print the first few rows of the GeoDataFrame



,id,English,Arabic,Arabic_2,German,English_2,Hebrew,geometry
0,None,Storks Tower,Burj Laklak,None,None,None,None,POINT (35.23671 31.78368)
1,None,St. Stephen's Gate,Bâb Sitti Maryam,Bāb Sitti Marjam,Stephansthor,None,None,POINT (35.23693 31.78085)
2,None,Gate of the Tribes\t\t\t\t,Bâb al Asbât\t,None,None,None,None,POINT (35.23693 31.78026)
3,None,Solomon's Throne,None,None,None,None,None,POINT (35.23706 31.77941)
4,None,Golden Gate,Bâb al Taûbe',None,None,None,None,POINT (35.23707 31.77897)


In [4]:
from fuzzywuzzy import fuzz
import pandas as pd

# Function to perform fuzzy matching
def fuzzy_match(toponym1, toponym2):
    return fuzz.token_set_ratio(str(toponym1), str(toponym2))

# Iterate through rows in json_df and find matches in gdf
matches = []
for index1, row1 in json_df.iterrows():
    place_in_json = row1['place']

    # Iterate through rows in gdf
    for index2, row2 in gdf.iterrows():
        # Iterate through language columns in gdf
        for lang_column in ["English", "Arabic", "English_2", "Arabic_2", "German", "Hebrew"]:
            place_in_gdf = row2[lang_column]

            # Skip if the language column is None
            if pd.notnull(place_in_gdf):
                similarity_score = fuzzy_match(place_in_json, place_in_gdf)
                
                # Check if similarity score is above the threshold
                if similarity_score >= 80:
                    matches.append({
                        'place_in_json': place_in_json,
                        'place_in_gdf': place_in_gdf,
                        'similarity_score': similarity_score
                    })
                    break  # Break out of the loop once a match is found in any language

# Create a dataframe from the list of matches
matches_df = pd.DataFrame(matches)

# Display or further process the matches_df as needed
print(matches_df)


                   place_in_json                     place_in_gdf  \
0                     Gethsemane                       Gethsemane   
1                           Zion                        Zion Gate   
2                           Zion                        Mt.  Zion   
3                  Damascus Gate          Damascus Gate\t\t\t\t\t   
4             St. Stephen's Gate               St. Stephen's Gate   
5             St. Stephen's Gate             St. Stephen's Church   
6             St. Stephen's Gate            St.  Stephen's Church   
7                    Golden Gate                      Golden Gate   
8                      Zion Gate                        Zion Gate   
9                     Jaffa Gate                       Jaffa Gate   
10                    Jaffa Gate      Police Station,  Jaffa Gate   
11                Pool of Siloam                   Pool of Siloam   
12                Pool of Siloam               Old Pool of Siloam   
13           Church of St. James  